<a href="https://colab.research.google.com/github/ViniciusSilveiraCampos/Analise-e-Classifica-o-de-Sentimentos/blob/main/Analise_Sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Iᴍᴘᴏʀᴛᴀᴄ̧ᴀ̃ᴏ

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import plotly.express as px
import plotly.graph_objects as ir
# Visualização dos dados

## Bᴀsᴇ ᴅᴇ ᴅᴀᴅᴏs

In [2]:
Base = pd.read_csv('emotion_sentimen_dataset.csv')

In [5]:
Base = Base[Base['Emotion'] != 'neutral']
# Eliminando a emoção que mais aparece

In [6]:
Base.Emotion.unique()
# Verificando as classes

#'ódio', 'raiva', 'amor', 'preocupação', 'alívio', 'felicidade', 'diversão',
# 'vazio', 'entusiasmo', 'tristeza', 'surpresa', 'tédio'

array(['hate', 'anger', 'love', 'worry', 'relief', 'happiness', 'fun',
       'empty', 'enthusiasm', 'sadness', 'surprise', 'boredom'],
      dtype=object)

In [7]:
# Exemplo de dados
classes = list(Base.Emotion.values)
label_encoder = LabelEncoder()

# Transformar as classes em números
numeric_classes = label_encoder.fit_transform(classes)

# Criar um dicionário de mapeamento de classe para número
class_to_number = dict(zip(classes, numeric_classes))
Base['Emotion'] = Base['Emotion'].map(class_to_number)

<ipython-input-7-232966c07370>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Base['Emotion'] = Base['Emotion'].map(class_to_number)


In [9]:
fig = px.histogram(Base.Emotion)
fig

In [11]:
Teste = Base[:500]
Treino = Base[500:9000]
# Dividindo a quantidade de treino, para não usar todos os dados da planilha

## Cʀɪᴀɴᴅᴏ ᴏ ᴠᴏᴄᴀʙᴜʟᴀʀɪᴏ ᴅᴇ ᴘᴀʟᴀᴠʀᴀs

In [13]:
train_data = []
test_data = []
# Organizando os dados em uma lista de truplas
numeros = len(Treino)
for a in range(0, len(Treino)):
  train_data.append((Treino.text.values[a], Treino.Emotion.values[a]))

for a in range(0, len(Teste)):
  test_data.append((Teste.text.values[a], Teste.Emotion.values[a]))

In [15]:
# Criar vocabulário único a partir dos dados de treinamento e teste
vocab = set()
for text, _ in train_data + test_data:
    vocab.update(text.lower().split())

# Mapear palavras para índices numéricos
word_to_index = {word: i for i, word in enumerate(vocab)}


In [16]:
import torch

# Função para converter texto em tensor
def text_to_tensor(text):
    tensor = torch.zeros(len(text), dtype=torch.long)
    for i, word in enumerate(text):
        tensor[i] = word_to_index[word]
    return tensor

# Converter dados de treinamento e teste em tensores
train_data_tensors = [(text_to_tensor(text.lower().split()), label) for text, label in train_data]
test_data_tensors = [(text_to_tensor(text.lower().split()), label) for text, label in test_data]


## Mᴏᴅᴇʟᴏ ᴅᴇ ʀᴇᴅᴇ ɴᴇᴜʀᴀʟ

In [21]:
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return output

# Parâmetros da rede
vocab_size = len(vocab)
embedding_size = 10
hidden_size = 32
output_size = 12  # Classificação em 12 categorias

# Criar uma instância do modelo
model = SimpleRNN(vocab_size, embedding_size, hidden_size, output_size)


In [22]:
model()

SimpleRNN(
  (embedding): Embedding(11765, 10)
  (rnn): RNN(10, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=12, bias=True)
)

## Tʀᴇɪɴᴀᴍᴇɴᴛᴏ ᴅᴏ ᴍᴏᴅᴇʟᴏ

In [23]:
# Definir função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Treinamento
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.
    for text_tensor, label in train_data_tensors:
        text_tensor = text_tensor


        optimizer.zero_grad()
        output = model(text_tensor.unsqueeze(0))  # Adiciona dimensão do lote
        loss = criterion(output, torch.tensor([label]))  # CrossEntropyLoss espera um tensor de rótulos
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('Época %3d: perda %.5f' % (epoch+1, running_loss/num_epochs))

Época   1: perda 375.38687
Época   2: perda 348.46961
Época   3: perda 317.55161
Época   4: perda 281.01966
Época   5: perda 245.11627
Época   6: perda 231.53782
Época   7: perda 201.61900
Época   8: perda 213.06634
Época   9: perda 197.21527
Época  10: perda 161.99705
Época  11: perda 138.05154
Época  12: perda 121.95957
Época  13: perda 118.62091
Época  14: perda 102.40621
Época  15: perda 96.22598
Época  16: perda 118.67216
Época  17: perda 104.03545
Época  18: perda 84.71824
Época  19: perda 100.32983
Época  20: perda 76.81089
Época  21: perda 94.83458
Época  22: perda 113.95098
Época  23: perda 89.27628
Época  24: perda 80.14441
Época  25: perda 70.70443
Época  26: perda 73.38363
Época  27: perda 57.62773
Época  28: perda 74.87809
Época  29: perda 66.97533
Época  30: perda 67.68919
Época  31: perda 51.82680
Época  32: perda 50.66704
Época  33: perda 53.85504
Época  34: perda 44.70095
Época  35: perda 43.94907
Época  36: perda 46.59680
Época  37: perda 46.71681
Época  38: perda 42.

## Aᴠᴀʟɪᴀᴄ̧ᴀ̃ᴏ ᴅᴀ ʀᴇᴅᴇ ɴᴇᴜʀᴀʟ

In [24]:
from sklearn.metrics import accuracy_score

# Avaliação
def evaluate(model, data_tensors):
    correct_predictions = []
    with torch.no_grad():
        for text_tensor, label in data_tensors:
            text_tensor = text_tensor #.to(device)
            output = model(text_tensor.unsqueeze(0))
            _, predicted = torch.max(output.data, 1)
            correct_predictions.append((predicted.item(), label))
    return correct_predictions

# Avaliar o modelo com os dados de teste
correct_predictions = evaluate(model, test_data_tensors)

# Separar previsões e rótulos verdadeiros
predicted_labels = [pred for pred, _ in correct_predictions]
true_labels = [true for _, true in correct_predictions]

# Calcular a acurácia
test_accuracy = accuracy_score(true_labels, predicted_labels)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.766


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [26]:
matriz = confusion_matrix(true_labels, predicted_labels)
matriz

array([[48,  1,  1,  1,  2,  2,  0,  6,  2,  2,  1],
       [ 2,  6,  1,  0,  0,  0,  0,  1,  1,  0,  2],
       [ 2,  1, 30,  1,  0,  1,  1,  0,  0,  0,  2],
       [ 0,  0,  1, 16,  1,  1,  0,  1,  1,  0,  1],
       [ 1,  2,  0,  1, 65,  2,  5,  7,  1,  0,  0],
       [ 2,  3,  0,  0,  0, 51,  0,  4,  2,  1,  1],
       [ 0,  1,  0,  0,  3,  0, 79,  4,  6,  0,  0],
       [ 3,  0,  1,  2,  2,  2,  1, 48,  1,  4,  0],
       [ 1,  0,  1,  0,  0,  0,  1,  1, 28,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0],
       [ 2,  2,  0,  3,  2,  0,  0,  0,  0,  1, 11]])

In [27]:
fig = px.imshow(matriz, color_continuous_scale='gray', text_auto=True)
fig.show()